In [10]:
import awswrangler as awr
import pandas as pd
import os
import openpyxl


caminho_query = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\query_pagamentos.sql"

#realizando a leitura da query
with open(caminho_query,'r') as arquivo_query:
    query = arquivo_query.read()

#transformando em dataframe (pandas) e executando a consulta no athena 
df_pagamentos = awr.athena.read_sql_query(query, database='silver')


In [11]:
df_pagamentos.head()

,cooperativa,associado,codigo_cadastro,ponteiro,numero_boleto,nosso_numero,historico,valor_titulo,data_pagamento,data_vencimento,data_emissao,dias_atraso,vendedor,grupo,aplicacao_financeira
0,Segtruck,TS CONSULTORIA EM TRANSPORTES LTDA,1,2455862,,,1,113400.00,2244-09-16,2024-09-16,2024-09-16,-80353,OUTROS,ADIANTAMENTOS,Adiantamento de Cliente
1,Segtruck,DIONEIO GONCALVES DA CRUZ & CIA LTDA -,5416892,635691,,,1,400.00,2201-12-03,2019-12-03,2019-12-03,-66474,OUTROS,RECEITAS NAO OPERACIONAIS,Recuperacoes de Dispendios
2,Segtruck,TK LOCACAO DE VEICULOS E MAQUINAS LTDA,210838,771812,,,1,4978.49,2200-04-23,2020-04-23,2020-04-23,-65743,OUTROS,RECEITAS,Receita de ALuguel de Imovel
3,Segtruck,COOPERATIVA DE TRABALHO DOS PROP. DE VEIC. CAR...,7668,176108,0,<NA>,1,223.76,2106-10-26,2106-10-26,2106-10-26,0,OUTROS,RECEITAS NAO OPERACIONAIS,Recuperacoes de Despesas
4,Segtruck,COOPERATIVA DE TRABALHO DOS PROP. DE VEIC. CAR...,7668,1086977,0,<NA>,1,150.00,2040-02-23,2040-02-23,2040-02-23,0,OUTROS,Padrão,Padrão- ALTERAR APLICACAO


In [15]:
df_pagamentos_dup = df_pagamentos[df_pagamentos['ponteiro'].duplicated(keep=False)]
df_pagamentos_baixados = df_pagamentos_dup.sort_values('data_pagamento', ascending=False).drop_duplicates('ponteiro', keep='first')


In [17]:
df_pagamentos_baixados['data_pagamento']=pd.to_datetime(df_pagamentos_baixados['data_pagamento'], format='%d/%m/%y')

In [21]:
df_pagamentos_baixados

,cooperativa,associado,codigo_cadastro,ponteiro,numero_boleto,nosso_numero,historico,valor_titulo,data_pagamento,data_vencimento,data_emissao,dias_atraso,vendedor,grupo,aplicacao_financeira
0,Segtruck,TS CONSULTORIA EM TRANSPORTES LTDA,1,2455862,,,1,113400.00,2244-09-16,2024-09-16,2024-09-16,-80353,OUTROS,ADIANTAMENTOS,Adiantamento de Cliente
1,Segtruck,DIONEIO GONCALVES DA CRUZ & CIA LTDA -,5416892,635691,,,1,400.00,2201-12-03,2019-12-03,2019-12-03,-66474,OUTROS,RECEITAS NAO OPERACIONAIS,Recuperacoes de Dispendios
2,Segtruck,TK LOCACAO DE VEICULOS E MAQUINAS LTDA,210838,771812,,,1,4978.49,2200-04-23,2020-04-23,2020-04-23,-65743,OUTROS,RECEITAS,Receita de ALuguel de Imovel
3,Segtruck,COOPERATIVA DE TRABALHO DOS PROP. DE VEIC. CAR...,7668,176108,0,<NA>,1,223.76,2106-10-26,2106-10-26,2106-10-26,0,OUTROS,RECEITAS NAO OPERACIONAIS,Recuperacoes de Despesas
5,Segtruck,COOPERATIVA DE TRABALHO DOS PROP. DE VEIC. CAR...,7668,2496185,,,1,3000.00,2025-12-11,2025-12-11,2024-12-11,0,OUTROS,ADIANTAMENTOS,Adiantamento de Cliente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415330,Segtruck,EDSON CARLOS GUARIENTE,14523,198292,0191888,0191888-5,4,757.10,2017-10-24,2018-09-25,2017-10-24,336,OUTROS,RECEITAS,Mensalidades/Benefícios
3415554,Segtruck,DANILO NUNES DA SILVA TRANSPORTES EIRELI ME,14411,195806,0191260,0191260-9,1,885.24,2017-10-20,2018-08-25,2017-10-20,309,Rafael Vieira,RECEITAS,Mensalidades/Benefícios
1711539,Segtruck,RIVA CORREA,8690,188348,0181791,0181791-9,1,181.77,2017-08-30,2018-08-01,2017-08-30,336,Tiago Americo de Oliveira,RECEITAS,Mensalidades/Benefícios
3417010,Segtruck,VALDAIR MICHELS,12769,198600,0174237,0174237-1,4,729.44,2017-07-24,2018-05-10,2017-07-24,290,OUTROS,RECEITAS,Mensalidades/Benefícios


In [29]:
hoje = pd.Timestamp.today()
um_ano_atras = hoje-pd.DateOffset(year=1)

df_data_especifica = df_pagamentos_baixados[
    (df_pagamentos_baixados['data_pagamento']<=hoje)
    &
    (df_pagamentos_baixados['data_pagamento']>=um_ano_atras)
    ]

df_data_especifica


,cooperativa,associado,codigo_cadastro,ponteiro,numero_boleto,nosso_numero,historico,valor_titulo,data_pagamento,data_vencimento,data_emissao,dias_atraso,vendedor,grupo,aplicacao_financeira
140,Stcoop,CONCRESIL CONCRETO E AREIA LTDA,11292,214895,1182213,1182213-0,1,1044.01,2025-02-26,2025-10-10,2025-02-26,226,Augustinho Pignatel,RECEITAS,Mensalidades/Benefícios
131,Stcoop,PAULO MARCOS PRIOR,8199,131808,1123620,1123620-4,4,1429.77,2025-02-26,2024-12-26,2024-05-24,-62,OUTROS,RECEITAS,Mensalidades/Benefícios
132,Stcoop,CONCRESIL CONCRETO E AREIA LTDA,11292,214887,1182118,1182118-4,1,250.00,2025-02-26,2025-03-20,2025-02-26,22,Augustinho Pignatel,RECEITAS,Adesões
133,Stcoop,PAULO MARCOS PRIOR,8199,131810,1123616,1123616-4,4,1429.77,2025-02-26,2025-02-25,2024-05-24,-1,OUTROS,RECEITAS,Mensalidades/Benefícios
134,Stcoop,CONCRESIL CONCRETO E AREIA LTDA,11292,214890,1182120,1182120-0,1,1044.01,2025-02-26,2025-05-12,2025-02-26,75,Augustinho Pignatel,RECEITAS,Mensalidades/Benefícios
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415330,Segtruck,EDSON CARLOS GUARIENTE,14523,198292,0191888,0191888-5,4,757.10,2017-10-24,2018-09-25,2017-10-24,336,OUTROS,RECEITAS,Mensalidades/Benefícios
3415554,Segtruck,DANILO NUNES DA SILVA TRANSPORTES EIRELI ME,14411,195806,0191260,0191260-9,1,885.24,2017-10-20,2018-08-25,2017-10-20,309,Rafael Vieira,RECEITAS,Mensalidades/Benefícios
1711539,Segtruck,RIVA CORREA,8690,188348,0181791,0181791-9,1,181.77,2017-08-30,2018-08-01,2017-08-30,336,Tiago Americo de Oliveira,RECEITAS,Mensalidades/Benefícios
3417010,Segtruck,VALDAIR MICHELS,12769,198600,0174237,0174237-1,4,729.44,2017-07-24,2018-05-10,2017-07-24,290,OUTROS,RECEITAS,Mensalidades/Benefícios


In [30]:

df_data_especifica = df_data_especifica[df_data_especifica['dias_atraso']<=0]
df_data_especifica['valor_titulo'].sum()

1959790515.7200005